In [1]:
import numpy as np
import pandas as pd

In [14]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);
dfDataChampionship = pd.read_excel("b2001.xlsx")

In [15]:
dfDataChampionship.head()

,Horario,Dia,Data,Clube 1,Clube 2,Placar,Vencedor,Rodada,Arena
0,14h30,Quarta-feira,01/08/2001,Portuguesa,Goiás,0x1,Goiás,1ª Rodada,Canindé
1,20h30,Quarta-feira,01/08/2001,Botafogo-RJ,América-MG,1x1,-,1ª Rodada,Maracanã
2,20h30,Quarta-feira,01/08/2001,Guarani,Palmeiras,1x3,Palmeiras,1ª Rodada,Brinco de Ouro
3,20h30,Quarta-feira,01/08/2001,Paraná,Cruzeiro,1x0,Paraná,1ª Rodada,Durival de Brito
4,20h30,Quarta-feira,01/08/2001,Internacional,Juventude,0x2,Juventude,1ª Rodada,Beira Rio


** Get all clubes for championship **



In [16]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([dfDataChampionship['Clube 1'].str.lower(), dfDataChampionship['Clube 2'].str.lower()], axis=1, keys=['Clubes'])

In [17]:
#Obtem todos os clubes 
cb = pd.DataFrame({'Clubes': clubs['Clubes'].unique()})

In [18]:
#Todos os clubes que participaram dp campeonato
cb

,Clubes
0,portuguesa
1,botafogo-rj
2,guarani
3,paraná
4,internacional
5,sport
6,bahia
7,botafogo-sp
8,gama
9,atlético-mg


In [19]:
def getNumberWinner(data, clube ):
    filter = data["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    filter1 = data["Clube 1"].str.lower() == clube
    filter2 = data["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    filter1 = data["Clube 1"].str.lower() == clube
    filter2 = data["Clube 2"].str.lower() == clube
    filter3 = data["Vencedor"].str.lower() == clube
    filter4 = (data["Clube 1"].str.lower() == clube) | (data["Clube 2"].str.lower() == clube)
    filter5 = data["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()    
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    filter1 = (data["Clube 1"].str.lower() == clube) | (data["Clube 2"].str.lower() == clube)
    filter2 = data["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    filter1 = (data["Clube 1"].str.lower() == clube) | (data["Clube 2"].str.lower() == clube)
    filter2 = (data["Vencedor"].str.lower() != clube) & (data["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    filter1  =  data["Clube 1"].str.lower() == clube
    filter2  =  data["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [20]:
dfTable = cb[["Clubes"]].copy()

for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = pd.np.nan
    
for index, row in dfTable.iterrows():
    gp, gc   =  getGP(dfDataChampionship, row["Clubes"] )
    
    dfTable.at[index, 'PG'] = getPoints(dfDataChampionship, row["Clubes"] )
    dfTable.at[index, 'J']  = getNumberDepartures(dfDataChampionship, row["Clubes"] )
    dfTable.at[index, 'V']  = getNumberWinner(dfDataChampionship, row["Clubes"] )    
    dfTable.at[index, 'E']  = getDraw(dfDataChampionship, row["Clubes"] )
    dfTable.at[index, 'D']  = getDefeats(dfDataChampionship, row["Clubes"] )
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc   

In [21]:
reorderlist = ['Atlético-pr', 'São Caetano', 'Fluminense', 'Atlético-mg', 'Grêmio', 'Ponte Preta',
               'São Paulo', 'Bahia', 'Internacional', 'Goiás', 'Vasco', 'Palmeiras', 'Portuguesa',
               'Paraná', 'Santos', 'Vitória', 'Coritiba', 'Corinthians', 'Guarani', 'Gama', 'Cruzeiro',
               'Juventude', 'Botafogo-rj', 'Flamengo', 'Santa Cruz', 'América-mg', 'Botafogo-sp', 'Sport']
listIndex   = []

#Processa a lista para gerar a lista de ordenação
for clube in reorderlist:
    for index, row in dfTable.iterrows():
        #Verifica se o valor de clube é um unicode, se não, convete isso.
        if type(clube) != unicode:
            c =  clube.decode('utf-8')
        else:
            c =  clube
        
        c  =  c.lower()
        
        #Verifica se o clube iterado é igual ao clube da lista, se sim
        #retorna seu indice
        if row['Clubes'] == c:
            listIndex.append( index )
            break

            
#Reordena a lista
dfTable  =  dfTable.reindex(index=listIndex, columns=["Clubes", "PG", "J", "V", "E", "D", "GP", "GC", "SG"])
#Com a lista ordenada, corrige os indices.
dfTable.index =  np.arange(1, len(dfTable)+1)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
1,atlético-pr,66.0,31.0,20.0,6.0,5.0,69.0,44.0,25.0
2,são caetano,63.0,31.0,19.0,6.0,6.0,52.0,31.0,21.0
3,fluminense,51.0,29.0,14.0,9.0,6.0,48.0,35.0,13.0
4,atlético-mg,52.0,29.0,16.0,4.0,9.0,54.0,36.0,18.0
5,grêmio,47.0,28.0,14.0,5.0,9.0,39.0,32.0,7.0
6,ponte preta,47.0,28.0,13.0,8.0,7.0,55.0,51.0,4.0
7,são paulo,46.0,28.0,13.0,7.0,8.0,49.0,36.0,13.0
8,bahia,46.0,28.0,13.0,7.0,8.0,43.0,38.0,5.0
9,internacional,43.0,27.0,13.0,4.0,10.0,40.0,40.0,0.0
10,goiás,39.0,27.0,12.0,3.0,12.0,38.0,32.0,6.0


** Na fonte de coleta dos dados os clubes Atlético-PR, Fluminense, Internacional, Flamengo apresentam valores divergentes com os dados processados, em sjua maioria u **